In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Import Rental Data


In [96]:
rental_data = pd.read_csv("../data/raw/rental_data.csv")

In [97]:
# get a new column year from the rental dataframe
rental_data['year'] = pd.DatetimeIndex(rental_data['date']).year
rental_data

,lat,lng,address,bed,bath,car,type,rented_price,date,year
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01,2023
1,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01,2023
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01,2023
...,...,...,...,...,...,...,...,...,...,...
3388731,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-12-01,2011
3388732,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-11-01,2011
3388733,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,280,2009-11-01,2009
3388734,-37.895171,144.759854,"53 BROADBEACH CIRCUIT, POINT COOK",4.0,2.0,2.0,Townhouse,420,2019-09-01,2019


In [99]:
rental_data[(rental_data['year'] >= 2012) & (rental_data['year'] <= 2016)].count()

lat             1079957
lng             1079957
address         1079957
bed             1079957
bath            1079957
car             1079957
type            1079957
rented_price    1079957
date            1079957
year            1079957
dtype: int64

In [4]:
rental_data = rental_data[rental_data['year'] >= 2017]

# Import price data

In [5]:
price_data_0 = pd.read_csv("../data/landing/oldlistings_buy_0.csv")
price_data_1 = pd.read_csv("../data/landing/oldlistings_buy_1.csv")
price_data_2 = pd.read_csv("../data/landing/oldlistings_buy_2.csv")
price_data_3 = pd.read_csv("../data/landing/oldlistings_buy_3.csv")
price_data_4 = pd.read_csv("../data/landing/oldlistings_buy_4.csv")
price_data_5 = pd.read_csv("../data/landing/oldlistings_buy_5.csv")
price_data_6 = pd.read_csv("../data/landing/oldlistings_buy_6.csv")

In [6]:
# combine all the data
price_data = pd.concat([price_data_0, price_data_1, price_data_2, price_data_3, price_data_4, price_data_5, price_data_6])

# extract the region from the address which is separated by the comma
price_data['region'] = price_data['address'].str.split(',').str[1]
price_data

,Unnamed: 0,lat,lng,rented_prices,address,meta_data,region
0,0,NaN,NaN,"[{'date': 'April 2013', 'price': '$340,00\u200...","𝟸𝟻-𝟸𝟽 𝙻ОՍIS RОΑD ​- 𝟷ST ΕSTΑTΕ, VΕNՍS BAY​","[{'label': 'bed', 'description': 'Bed', 'quant...",VΕNՍS BAY​
1,1,NaN,NaN,"[{'date': 'April 2013', 'price': '$４𝟫𝟫,OOO'}, ...","85 PANDORA AVENUE - 1ST ESTATE, VENUS BAY","[{'label': 'bed', 'description': 'Bed', 'quant...",VENUS BAY
2,2,NaN,NaN,"[{'date': 'April 2013', 'price': <span>$𝟨𝟻,OOO...","𝟨-𝟪 В𝙻ΑСK AVEN​UE, VENUS BAY","[{'label': 'type', 'description': 'Category', ...",VENUS BAY
3,3,NaN,NaN,"[{'date': 'April 2013', 'price': '$3４𝟫,OOO'}, ...","2 SYDNEY COURT, VΕNՍS ВΑY","[{'label': 'bed', 'description': 'Bed', 'quant...",VΕNՍS ВΑY
4,4,NaN,NaN,"[{'date': 'April 2013', 'price': '$25\u200b0,0...","119 JUPITER BLVD, VΕNՍS ВΑY","[{'label': 'bed', 'description': 'Bed', 'quant...",VΕNՍS ВΑY
...,...,...,...,...,...,...,...
176299,176299,NaN,NaN,"[{'date': 'December 2017', 'price': 'Unde\\u20...","3O VIСTОRKINԌ DRIV​E, PОINT СООK","[{'label': 'bed', 'description': 'Bed', 'quant...",PОINT СООK
176300,176300,NaN,NaN,"[{'date': 'December 2017', 'price': <span>Co​n...","60 SEAGRASS CRESCENT, PОINT СООK","[{'label': 'bed', 'description': 'Bed', 'quant...",PОINT СООK
176301,176301,NaN,NaN,"[{'date': 'December 2017', 'price': 'Under Con...","2​6 PΑ𝙻ΑСΕ RОΑD, POINT​ СООK","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT​ СООK
176302,176302,NaN,NaN,"[{'date': 'December 2017', 'price': 'SOLD'}, {...","𝟸𝟽𝟻 PОINT COO​K ROAD​, POINT COOK","[{'label': 'land', 'description': 'Land', 'qua...",POINT COOK


# Import population and income data

In [7]:
population_data = pd.read_csv("../data/raw/population/population-01-23.csv")
income_data_2 = pd.read_csv("../data/raw/income/income-17-21.csv")

population_data

,sa2_code,sa2_name,year,population,geometry
0,201011001,Alfredton,2001,5756.0,POLYGON ((143.78282104711133 -37.5666578080732...
1,201011002,Ballarat,2001,11497.0,POLYGON ((143.81896375194268 -37.5558184903035...
2,201011005,Buninyong,2001,5320.0,POLYGON ((143.8417079819547 -37.61596158212406...
3,201011006,Delacombe,2001,4154.0,POLYGON ((143.7505006002539 -37.59119089617515...
4,201011007,Smythes Creek,2001,3317.0,POLYGON ((143.7329551500814 -37.62333175526272...
...,...,...,...,...,...
12001,217031476,Otway,2023,3983.0,MULTIPOLYGON (((143.40263218222083 -38.7815167...
12002,217041477,Moyne - East,2023,7132.0,POLYGON ((142.41438437646818 -38.0930256782602...
12003,217041478,Moyne - West,2023,10148.0,MULTIPOLYGON (((142.00870118938005 -38.4171452...
12004,217041479,Warrnambool - North,2023,22762.0,POLYGON ((142.4366836580644 -38.35544112113083...


In [8]:
# change the type of income_data_2 to float
income_data_2['median_income'] = income_data_2['median_income'].str.replace(',', '').astype(float)

In [9]:
population_data["geometry"] = population_data["geometry"].apply(wkt.loads)
income_data_2["geometry"] = income_data_2["geometry"].apply(wkt.loads)

gdf_population = gpd.GeoDataFrame(population_data, geometry="geometry", crs="EPSG:4326")
gdf_income_2 = gpd.GeoDataFrame(income_data_2, geometry="geometry", crs="EPSG:4326")

gdf_rental = gpd.GeoDataFrame(rental_data, geometry=gpd.points_from_xy(rental_data["lng"], rental_data["lat"]), crs="EPSG:4326")

# join rental data with population and income

In [10]:
# left sptail join rental lat lon on population geometry column

joined_gdf = gpd.sjoin(gdf_rental,gdf_population, how="left", predicate = "within", on_attribute=["year"])


In [11]:
joined_gdf = joined_gdf.dropna()

In [12]:
# let join_gdf1 be the df that only has year 2012-2016 and join_gdf2 be the df that only has year 2017-2021

joined_gdf2 = joined_gdf[joined_gdf["year"] > 2016]



In [13]:
joined_gdf2.drop(columns=["index_right"], inplace=True)
joined_gdf_new2 = gpd.sjoin(joined_gdf2,gdf_income_2, how="left", predicate = "within", on_attribute=["year"])
joined_gdf_new2

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code_left,sa2_name_left,population,index_right,sa2_code_right,sa2_name_right,median_income
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,16098.0,2364.0,206041505.0,Melbourne CBD - West,39300.0
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
5,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,650,2023-06-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388726,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-09-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388727,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-08-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388728,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-07-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388729,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,365,2018-06-01,2018,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9678.0,889.0,213051581.0,Point Cook - North West,57001.0


In [14]:
joined_gdf = joined_gdf_new2

In [15]:
joined_gdf = joined_gdf.dropna()

In [16]:
joined_gdf.drop(columns = "index_right",inplace=True)

C:\Users\Richard\AppData\Local\Temp\ipykernel_5128\3371548854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_gdf.drop(columns = "index_right",inplace=True)


In [18]:
joined_gdf[joined_gdf["year"] > 2021]

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code_left,sa2_name_left,population,sa2_code_right,sa2_name_right,median_income


# Import cpidata, unemployement rate and housing interest data

In [43]:
cpi_data = pd.read_csv("../data/curated/cpi_data.csv")
unemployment_data = pd.read_csv("../data/curated/yearly_avg_unemployment.csv")
housing_interest_data = pd.read_csv("../data/curated/housing_interest.csv")

In [44]:
# change the time column to year only
housing_interest_data["time"] = housing_interest_data["time"].apply(lambda x: x[:4])

In [45]:
# aggregate the housing_interest_data by year
housing_interest_data = housing_interest_data.groupby("time")["interest_rate"].mean().reset_index()
housing_interest_data

,time,interest_rate
0,2019,4.171667
1,2020,3.656667
2,2021,3.415000
3,2022,4.308333
4,2023,6.374167
5,2024,6.661667


In [46]:
housing_interest_data.rename(columns={"time":"year"}, inplace=True)

In [47]:
unemployment_data.rename(columns = {"Yearly Average Seasonally Adjusted (%)":"unemployment_rate"}, inplace=True)

In [48]:
cpi_data.drop(columns = ["time","Change from previous quarter (%)"] ,inplace=True)
cpi_data.rename(columns = {"Annual change (%)" : "cpi" }, inplace=True)
# add a row in unemployment_data that 2024 has rate 4.2
new_row = pd.DataFrame({"year": [2024], "cpi": [3.8]})
cpi_data = pd.concat([cpi_data, new_row], ignore_index=True)
cpi_data

,cpi,year
0,1.7,2014
1,1.7,2015
2,1.5,2016
3,1.9,2017
4,1.8,2018
5,1.8,2019
6,0.9,2020
7,3.5,2021
8,7.8,2022
9,4.1,2023


In [49]:
# add a row in unemployment_data that 2021 has rate 4.2
new_row = pd.DataFrame({"year": [2021], "unemployment_rate": [4.2]})
unemployment_data = pd.concat([unemployment_data, new_row], ignore_index=True)
unemployment_data

,year,unemployment_rate
0,2014,6.216667
1,2015,6.058333
2,2016,5.716667
3,2017,5.608333
4,2018,5.283333
5,2019,5.175000
6,2020,5.166667
7,2022,3.611111
8,2023,3.691667
9,2024,4.014286


# model median income using time series

In [26]:
unemployment_data

,year,unemployment_rate
0,2014,6.216667
1,2015,6.058333
2,2016,5.716667
3,2017,5.608333
4,2018,5.283333
5,2019,5.175000
6,2020,5.166667
7,2022,3.611111
8,2023,3.691667
9,2024,4.014286


In [81]:
# join the cpi_data and unemployment_data on year
joined_gdf_with_cpi = pd.merge(joined_gdf, cpi_data, on="year", how="left")
joined_gdf_with_cpi_and_unem = pd.merge(joined_gdf_with_cpi, unemployment_data, on="year", how="left")

In [95]:
joined_gdf_with_cpi_and_unem.count()

bed                  1132493
bath                 1132493
car                  1132493
type                 1132493
rented_price         1132493
year                 1132493
geometry             1132493
population           1132493
region               1132493
median_income        1132493
cpi                  1132493
unemployment_rate    1132493
dtype: int64

In [82]:
joined_gdf_with_cpi_and_unem.drop(columns = ["lat","lng","address","sa2_code_left","sa2_name_left","sa2_code_right","date"], inplace=True)

In [83]:
joined_gdf_with_cpi_and_unem.rename(columns={"sa2_name_right":"region"},inplace = True)

In [84]:
# find the unique values of the region column
unique_region = joined_gdf_with_cpi_and_unem["region"].unique()

In [85]:
median_income_pred = pd.DataFrame(columns=["year","region","median_income"])

In [86]:
for j in range(2022,2025):
    for i in range(0,len(unique_region)):
        
        region_0 = joined_gdf_with_cpi_and_unem[joined_gdf_with_cpi_and_unem["region"] == unique_region[i]]
        X = region_0[["cpi","year","unemployment_rate"]]
        y = region_0["median_income"]

        model = LinearRegression()
        model.fit(X,y)

        data_2022 = [cpi_data[cpi_data["year"] == j]["cpi"].values[0],j,unemployment_data[unemployment_data["year"] == j]["unemployment_rate"].values[0]]

        new_row = pd.DataFrame({"year": [j], "region": [unique_region[i]], "median_income": model.predict([data_2022])[0]})

        median_income_pred = pd.concat([median_income_pred, new_row], ignore_index=True)
    print(j)
#median_income_pred

C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Temp\ipykernel_5128\1485602109.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  median_income_pred = pd.concat([median_income_pred, new_row], ignore_index=True)
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.

2022


C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493

2023


C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493

2024


C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493

In [87]:
median_income_pred.rename(columns={"median_income":"median_pred_income"},inplace=True)

In [88]:
income_data_2.rename(columns={"sa2_name":"region"}, inplace=True)
income_data_2.head(1)

# join the median_income_pred with income_data_2 on region
median_income_pred = pd.merge(median_income_pred, income_data_2, on="region", how="left")
median_income_pred

,year_x,region,median_pred_income,sa2_code,geometry,year_y,median_income
0,2022,Melbourne CBD - West,45282.982949,206041505,POLYGON ((144.95279813819363 -37.8161049267206...,2017,27991.0
1,2022,Melbourne CBD - West,45282.982949,206041505,POLYGON ((144.95279813819363 -37.8161049267206...,2018,30439.0
2,2022,Melbourne CBD - West,45282.982949,206041505,POLYGON ((144.95279813819363 -37.8161049267206...,2019,31746.0
3,2022,Melbourne CBD - West,45282.982949,206041505,POLYGON ((144.95279813819363 -37.8161049267206...,2020,32044.0
4,2022,Melbourne CBD - West,45282.982949,206041505,POLYGON ((144.95279813819363 -37.8161049267206...,2021,39300.0
...,...,...,...,...,...,...,...
7720,2024,Upper Yarra Valley,39860.000000,204011061,POLYGON ((145.80700991628774 -37.6748974656811...,2017,39089.0
7721,2024,Upper Yarra Valley,39860.000000,204011061,POLYGON ((145.80700991628774 -37.6748974656811...,2018,43740.0
7722,2024,Upper Yarra Valley,39860.000000,204011061,POLYGON ((145.80700991628774 -37.6748974656811...,2019,41375.0
7723,2024,Upper Yarra Valley,39860.000000,204011061,POLYGON ((145.80700991628774 -37.6748974656811...,2020,39982.0


In [89]:
median_income_pred.drop(columns=["year_y","median_income"], inplace=True)
median_income_pred.rename(columns={"year_x":"year"}, inplace=True)

In [91]:
income_data_2

,sa2_code,region,geometry,year,median_income
0,206071139,Abbotsford,POLYGON ((144.99254539277322 -37.8024856893544...,2017,58219.0
1,210011226,Airport West,POLYGON ((144.867064292942 -37.724714940312566...,2017,53423.0
2,206051128,Albert Park,POLYGON ((144.96766973330057 -37.8373669078686...,2017,65352.0
3,204011054,Alexandra,POLYGON ((145.59015065379967 -37.2247702673797...,2017,36773.0
4,201011001,Alfredton,POLYGON ((143.78282104711133 -37.5666578080732...,2017,50596.0
...,...,...,...,...,...
2590,205051104,Yarram,MULTIPOLYGON (((146.6219024767579 -38.75020696...,2021,41677.0
2591,213031352,Yarraville,POLYGON ((144.85914995429522 -37.8176431203511...,2021,76253.0
2592,216021414,Yarrawonga,POLYGON ((146.00050823019788 -36.0087679689967...,2021,46794.0
2593,215011394,Yarriambiack,POLYGON ((142.2390036238344 -35.99787479520491...,2021,48604.0


In [93]:
median_income_pred.drop_duplicates(inplace=True)
median_income_pred.rename(columns={"median_pred_income":"median_income"}, inplace=True)

# join the median_income_pred with income_data_2 on region
income_data_full = pd.concat([income_data_2, median_income_pred], ignore_index=True)
income_data_full

,sa2_code,region,geometry,year,median_income
0,206071139,Abbotsford,POLYGON ((144.99254539277322 -37.8024856893544...,2017,58219.000000
1,210011226,Airport West,POLYGON ((144.867064292942 -37.724714940312566...,2017,53423.000000
2,206051128,Albert Park,POLYGON ((144.96766973330057 -37.8373669078686...,2017,65352.000000
3,204011054,Alexandra,POLYGON ((145.59015065379967 -37.2247702673797...,2017,36773.000000
4,201011001,Alfredton,POLYGON ((143.78282104711133 -37.5666578080732...,2017,50596.000000
...,...,...,...,...,...
4135,211031451,Croydon - West,POLYGON ((145.28818275389443 -37.7809559896419...,2024,57938.813288
4136,215031402,Kerang,"POLYGON ((143.89191430851 -35.735210753426436,...",2024,50765.394964
4137,213041574,Fraser Rise - Plumpton,POLYGON ((144.67057300877906 -37.7078414069349...,2024,59227.000000
4138,205041096,Morwell,POLYGON ((146.36524250548774 -38.2250693398236...,2024,40893.000000


In [94]:
income_data_full.to_csv("../data/curated/predicted_median_income.csv")

# Preprocess price data

In [ ]:
def process_df(data_df):
    def find_meta(xs,label, default=None, conversion=lambda x: x):
        for x in xs:
            if x['label'] == label:
                return conversion(x['quantity'])
        return default

    pattern1 = r"\$(\d+)\s*(per week|pw)" 
    
    data = data_df
    
    # extract rented prices to multiple rows
    pdf2 = pd.DataFrame(data['rented_prices'].to_list(),index=data.index).stack()
    pdf2 = pdf2.reset_index()
    pdf2.rename(columns={'level_0':'index',0:'rented_price_dict'},inplace=True)
    pdf2.drop(columns='level_1',inplace=True)
    pdf = pdf2.join(data, on='index', how='left')
    
    # meta data
    pdf['bed'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'bed',np.nan))
    pdf['bath'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'bath',np.nan))
    pdf['car'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'car',np.nan))
    pdf['land'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'land',np.nan))
    pdf['type'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'type',"No type"))
    
    pdf.drop(columns=['meta_data','rented_prices','index'],inplace=True)

    # extract rented price to multiple columns
    pdf["rented_price"] = pdf["rented_price_dict"].apply(lambda x: re.search(pattern1, x['price']).group(1) if re.search(pattern1, x['price']) else None)
    pdf["date"] = pdf["rented_price_dict"].apply(lambda x: x['date'])
    return pdf